In [1]:
import numpy as np
from tensorflow.keras import Model as Model_
from tensorflow.keras.layers import Input, PReLU, LeakyReLU, MaxPooling2D, Dropout, concatenate, UpSampling2D, ReLU, Conv2D, Flatten, Reshape, Conv1D, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Conv2DTranspose, Dense, BatchNormalization
from tensorflow.keras.datasets import mnist, cifar10
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import pandas as pd


# torch.manual_seed(0) # Set for testing purposes, please do not change!

print(tf.__version__)

2.9.1


In [2]:
direction_csv_file = pd.read_csv("Directions 1.54138969.csv", header=None)

In [3]:
direction_csv_file

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,473.6836,444.9424,500.9961,448.0299,479.8393,530.7856,506.2184,622.5688,493.6608,621.9954,...,515.4715,456.4298,515.4715,456.4298,499.2511,448.2281,515.0607,479.1209,515.0607,479.1209
1,473.6876,444.9526,500.9847,448.0307,479.8167,530.7855,506.2182,622.5726,493.6592,621.9958,...,514.9991,456.2006,514.9991,456.2006,498.6736,447.8242,514.2874,478.6892,514.2874,478.6892
2,473.6931,444.9663,500.9775,448.0403,479.7976,530.7957,506.2265,622.5868,493.6538,621.9990,...,514.7042,456.0592,514.7042,456.0592,498.3504,447.4883,513.7126,478.3876,513.7126,478.3876
3,473.7039,444.9854,500.9819,448.0659,479.7655,530.8015,506.2534,622.5867,493.6528,622.0079,...,514.5616,455.9726,514.5616,455.9726,498.1750,447.2817,513.3622,478.2127,513.3622,478.2127
4,473.7261,445.0067,501.0016,448.0924,479.7466,530.8076,506.2579,622.5981,493.6506,622.0058,...,514.6009,455.9780,514.6009,455.9780,498.1767,447.2362,513.2590,478.2043,513.2590,478.2043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,508.8683,435.9992,531.6415,432.5145,531.7281,525.6750,536.5560,619.7991,507.8583,621.3291,...,552.5535,450.4184,552.5535,450.4184,535.6805,447.9043,557.0486,475.3463,557.0486,475.3463
1379,508.8894,435.9756,531.7077,432.5069,531.7758,525.6630,536.5682,619.7924,507.8712,621.3207,...,551.9902,450.3370,551.9902,450.3370,535.2592,447.4315,556.0175,475.3270,556.0175,475.3270
1380,508.9090,435.9860,531.6987,432.5204,531.7960,525.6733,536.5767,619.8024,507.8655,621.3173,...,551.3666,450.2951,551.3666,450.2951,534.9061,446.8693,554.7869,475.3963,554.7869,475.3963
1381,508.9323,435.9888,531.6935,432.5264,531.8196,525.6773,536.5802,619.8096,507.8681,621.3137,...,550.6077,450.2511,550.6077,450.2511,534.4236,446.4702,553.4522,475.5750,553.4522,475.5750


In [226]:
import os
import re

class Dataset_Preprocessing:
    def __init__(self, dir_path, include_dimension = 2, sample_size = 50, total_classes = 17):
        self.dir_path = dir_path
        self.include_dimension = include_dimension
        self.sample_size = sample_size
        self.total_classes = total_classes
        self.internal_folders = ['S1','S5','S6','S7','S8','S9','S11']
    
    def read_dataset(self):
        try:
            activity_vector = {}
            sampled_data = {}
            data_folder = 'Poses_D2_Positions' if self.include_dimension == 2 else 'Poses_D3_Positions'
            if not os.path.exists(self.dir_path):
                print('The Data Directory Does not Exist!')
                return None
            
            for fld in self.internal_folders:
                for file in os.listdir(os.path.join(self.dir_path, fld, data_folder)):
                    data = pd.read_csv(os.path.join(self.dir_path, fld, data_folder, file), header=None)
                    activity = self.__extract_activity(file)
                    if activity not in activity_vector:
                        total_keys = len(activity_vector.keys())
                        activity_vector[activity] = np.zeros(self.total_classes)
                        activity_vector[activity][total_keys] = 1
                    vector = activity_vector[activity]
                    if (fld,activity) in sampled_data:
                        sampled_data[(fld,activity)].append(self.__group_samples(data, self.sample_size, vector))
                    else:
                        sampled_data[(fld,activity)] = self.__group_samples(data, self.sample_size, vector)
            return sampled_data
        except Exception as e:
            print(e)
    
    def __extract_activity(self, filename):
        try:
            name = os.path.splitext(filename)[0]
            activity = re.sub('[^A-Za-z]+' , '' , name)
            return activity
        except Exception as e:
            print(e)
    
    def __group_samples(self, dataset, sample_size, activity):
        try:
            if not isinstance(dataset, pd.DataFrame):
                print('Expecting Pandas Dataframe, but got {}'.format(type(dataset)))
                return None
            
            grouped_rows = []
            grouped_rows_activity = []
            starting_index = 0
            for i in range(0, dataset.shape[0]//sample_size):
                gr = dataset.iloc[starting_index:starting_index + sample_size,:]
                grouped_rows.append(np.asarray(gr))
                starting_index += sample_size
            
            for grp in grouped_rows:
                ngrp = []
                for row in grp:
                    ngrp.append(row.tolist() + activity.tolist())
                grouped_rows_activity.append(ngrp)
            return grouped_rows_activity
        except Exception as e:
            print(e)

In [227]:
sampled_data = Dataset_Preprocessing('H3.6csv').read_dataset()

In [4]:
#Convert to numpy array
direction_data_array = np.asarray(direction_csv_file)

In [5]:
class GloGen_Bidirectional_RNN_Encoder(Model_):
  def __init__(self, num_recurrent_neurons=200):
    super(GloGen_Bidirectional_RNN_Encoder, self).__init__()

    #Return Sequances=True to assure return of output corresponding to each timestep
    self.recurrent_layer = LSTM(num_recurrent_neurons, return_sequences=True)

  def call(self, input_x):
    output = self.recurrent_layer(input_x)
    #output = tf.one_hot(tf.argmax(output, axis = 1), depth=3)
    return output

In [6]:
#Not sure of the output shape from the glonet
class GloGen_Bidirectional_RNN_decoder(Model_):
  def __init__(self, num_recurrent_neurons=64):
    super(GloGen_Bidirectional_RNN_decoder, self).__init__()
    #Return Sequances=True to assure return of output corresponding to each timestep
    self.recurrent_layer = LSTM(num_recurrent_neurons, return_sequences=True)

  def call(self, encoder_output):
    glogen_output = self.recurrent_layer(encoder_output)
    #output = tf.one_hot(tf.argmax(output, axis = 1), depth=3)
    return glogen_output

In [7]:
class LocGen_Bidirectional_RNN_encoder(Model_) :
  def __init__(self, num_recurrent_neurons=200) :
    super(LocGen_Bidirectional_RNN_encoder, self).__init__()
    #Return Sequances=True to assure return of output corresponding to each timestep
    self.recurrent_layer = LSTM(num_recurrent_neurons, return_sequences=True)


  def call(self, interpolated_output) :
    predictions = self.recurrent_layer(interpolated_output)
    return predictions

In [8]:
class LocGen_Bidirectional_RNN_decoder(Model_) :
  def __init__(self, num_recurrent_neurons=200) :
    super(LocGen_Bidirectional_RNN_decoder, self).__init__()
    #Return Sequances=True to assure return of output corresponding to each timestep
    self.recurrent_layer = LSTM(num_recurrent_neurons, return_sequences=True)

  def call(self, encoder_output) :
    final_predictions = self.recurrent_layer(encoder_output)
    return final_predictions

In [9]:
glogen_bidirectional_RNN_Encoder = GloGen_Bidirectional_RNN_Encoder()

In [10]:
#Reshaping direction data array
direction_data_array = np.reshape(direction_data_array[:100], (10, 10, 64))

In [11]:
#Should have diminsions 100x60x64 as batch_siZe x timesteps x features
direction_data_array.shape

(10, 10, 64)

In [12]:
#simulating applying the glogen encoder on an input batch with batch size "10"
output_encoder = glogen_bidirectional_RNN_Encoder(direction_data_array)

In [13]:
#shape of 10x200 predictions for each sample inside the batch
output_encoder.shape

TensorShape([10, 10, 200])

In [14]:
#Simulating the output shape for the decoder
glogen_decoder = GloGen_Bidirectional_RNN_decoder()

In [15]:
glogen_sparse_output_predictions = glogen_decoder(output_encoder)

In [16]:
#Prediction for 32 joints position for the next 10 timesteps for each sample inside the batch
glogen_sparse_output_predictions.shape

TensorShape([10, 10, 64])

In [17]:
#returns a list of the interpolated frames between x0 and x1
def interpolateFrames(glogen_sparse_output_predictions, num_of_frames=5) :
  batch_size = glogen_sparse_output_predictions.shape[0]
  timesteps = glogen_sparse_output_predictions.shape[1]
  features = glogen_sparse_output_predictions.shape[2]
  interpolated_frames = np.zeros((batch_size, timesteps, num_of_frames, features))
  for batch in range(glogen_sparse_output_predictions.shape[0]) :
    for t in range(glogen_sparse_output_predictions.shape[1]-1) :
      for j in range(num_of_frames) :
        X_i0 = glogen_sparse_output_predictions[batch, t]
        X_i1 = glogen_sparse_output_predictions[batch, t+1]
        
        alpha_j = j/num_of_frames
        current_frame = alpha_j*X_i0 + (1-alpha_j)*X_i1
        interpolated_frames[batch, t, j] = current_frame

  return interpolated_frames 

In [18]:
interpolated_frames = interpolateFrames(glogen_sparse_output_predictions)

In [19]:
#Shape of interpolated frames should be 10(batch size) x 10(timesteps)
#x 5(num of interpolated frames between these timesteps) x 64(num_features)
interpolated_frames.shape

(10, 10, 5, 64)

In [20]:
#timesteps and interpolated frames diminsion can be flattened to 10(batch size) x 50(dense frames) x 64(features)
interpolated_frames = np.reshape(interpolated_frames, (10, 50, 64))

In [21]:
#Simulating the locgen phase with batches as well
locgen_encoder = LocGen_Bidirectional_RNN_encoder()
locgen_encoder_output = locgen_encoder(interpolated_frames)
print("loc gen encoder output is ", locgen_encoder_output.shape)

locgen_decoder = LocGen_Bidirectional_RNN_decoder()
final_predictions = locgen_decoder(locgen_encoder_output)

print("final predictions shape is ", final_predictions.shape)

loc gen encoder output is  (10, 50, 200)
final predictions shape is  (10, 50, 200)
